In [ ]:

from google.colab import files
files.upload()


import os
os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


!pip install -q kaggle
!kaggle datasets download -d akshitgupta146/plant-disease-dataset-classification
!unzip -q -o plant-disease-dataset-classification.zip -d plant_disease_dataset


import os
print("Contents of extracted folder:", os.listdir("plant_disease_dataset"))


from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_path = 'plant_disease_dataset/PlantVillage'

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    base_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


model.fit(train_generator, epochs=5, validation_data=validation_generator)


from sklearn.metrics import classification_report, accuracy_score, f1_score
import numpy as np

validation_generator.reset()
y_pred_probs = model.predict(validation_generator, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = validation_generator.classes
class_names = list(validation_generator.class_indices.keys())

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='macro')

print(f"\n✅ Accuracy: {acc:.4f}")
print(f"✅ F1 Score (macro): {f1:.4f}")
print("\n📋 Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))


from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()
img_path = list(uploaded.keys())[0]

img = load_img(img_path, target_size=(224, 224))
img_array = img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions, axis=1)[0]
predicted_label = class_names[predicted_class_index]

print(f"\n🔍 Predicted Disease: {predicted_label}")


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/akshitgupta146/plant-disease-dataset-classification
License(s): apache-2.0
plant-disease-dataset-classification.zip: Skipping, found more recently modified local copy (use --force to force download)
Contents of extracted folder: ['PlantVillage']
Found 1722 images belonging to 3 classes.
Found 430 images belonging to 3 classes.


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │    11,944,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,963,587 (45.64 MB)

 Trainable params: 11,963,587 (45.64 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 137s 3s/step - accuracy: 0.6095 - loss: 2.3945 - val_accuracy: 0.8674 - val_loss: 0.3312
Epoch 2/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - accuracy: 0.9175 - loss: 0.2214 - val_accuracy: 0.9209 - val_loss: 0.2216
Epoch 3/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.9694 - loss: 0.1018 - val_accuracy: 0.9465 - val_loss: 0.1470
Epoch 4/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 149s 2s/step - accuracy: 0.9890 - loss: 0.0380 - val_accuracy: 0.8837 - val_loss: 0.2550
Epoch 5/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.9779 - loss: 0.0602 - val_accuracy: 0.9419 - val_loss: 0.1906
14/14 ━━━━━━━━━━━━━━━━━━━━ 10s 679ms/step

✅ Accuracy: 0.4047
✅ F1 Score (macro): 0.2977

📋 Classification Report:
                       precision    recall  f1-score   support

Potato___Early_blight       0.41      0.40      0.41       200
 Potato___Late_blight       0.43      0.47      0.45       200
     Potato___healthy       0.05      0.03      0.04        30

 

Saving leaf31.jpeg to leaf31 (1).jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step

🔍 Predicted Disease: Potato___Early_blight
